In [422]:
import pandas as pd
import numpy as np # type: ignore
from pybaseball import *
import statsapi
import os

from pull_data import *
%load_ext autoreload
%autoreload 2
import requests

from pybaseball import cache
cache.enable()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [423]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

team_ids = [138, 140, 145, 110, 111, 144, 158, 143, 133, 141, 108, 114, 115, 121, 137, 146, 116, 118, 139, 117, 113, 112, 142, 136, 120, 109, 134, 135, 147, 119]

In [424]:
# Update roster player ids
reload = True
if reload:
    player_id_mapping = map_player_ids(team_ids)
else:
    player_id_mapping = pd.read_csv('player_id_mapping.csv')

No identically matched names found! Returning the 5 most similar names.
Victor Scott
No identically matched names found! Returning the 5 most similar names.
Luis Robert
Michael Taylor
No identically matched names found! Returning the 5 most similar names.
Enyel De Los Santos
Michael Harris
Ronald Acuña
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
Vladimir Guerrero
Luis L. Ortiz
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
Jung Hoo Lee
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
Bob

In [413]:
player_id_mapping

,name,abbrev,clean_name,clean_abbrev,team,player_id
0,Alec Burleson,A. Burleson,Alec Burleson,A. Burleson,Cardinals,676475
1,Andre Pallante,A. Pallante,Andre Pallante,A. Pallante,Cardinals,669467
2,Brendan Donovan,B. Donovan,Brendan Donovan,B. Donovan,Cardinals,680977
3,Erick Fedde,E. Fedde,Erick Fedde,E. Fedde,Cardinals,607200
4,Iván Herrera,I. Herrera,Ivan Herrera,I. Herrera,Cardinals,671056
...,...,...,...,...,...,...
775,Teoscar Hernández,T. Hernández,Teoscar Hernandez,T. Hernandez,Dodgers,606192
776,Tommy Edman,T. Edman,Tommy Edman,T. Edman,Dodgers,669242
777,Tony Gonsolin,T. Gonsolin,Tony Gonsolin,T. Gonsolin,Dodgers,664062
778,Will Smith,W. Smith,Will Smith,W. Smith,Dodgers,519293


In [414]:
from datetime import date

# Get today's date in YYYY-MM-DD format
check_yesterday = False
days_offset = 0
today = (date.today() + pd.DateOffset(days=days_offset)).strftime('%Y-%m-%d')
yesterday = (date.today() + pd.DateOffset(days=days_offset - 1)).strftime('%Y-%m-%d')
if check_yesterday:
    today = yesterday

# Get today's schedule
games = statsapi.schedule(start_date=today, end_date=today)

# Loop through games and extract probable pitchers
for game in games:
    home_team = game['home_name']
    away_team = game['away_name']
    home_pitcher = game.get('home_probable_pitcher', 'TBD')
    away_pitcher = game.get('away_probable_pitcher', 'TBD')
    
    print(f"{away_team} ({away_pitcher}) @ {home_team} ({home_pitcher})")

Miami Marlins (Edward Cabrera) @ Tampa Bay Rays (Zack Littell)
Philadelphia Phillies (Joe Ross) @ Pittsburgh Pirates (Bailey Falter)
Texas Rangers (Patrick Corbin) @ Washington Nationals (Michael Soroka)
Boston Red Sox (Walker Buehler) @ New York Yankees (Will Warren)
Arizona Diamondbacks (Eduardo Rodriguez) @ Cincinnati Reds (Nick Lodolo)
Chicago Cubs (Ben Brown) @ Detroit Tigers (Tarik Skubal)
Houston Astros (Colton Gordon) @ Cleveland Guardians (Logan Allen)
Kansas City Royals (Seth Lugo) @ Chicago White Sox (Davis Martin)
Toronto Blue Jays (Paxton Schultz) @ Minnesota Twins (Bailey Ober)
San Diego Padres (Randy Vásquez) @ Milwaukee Brewers (Chad Patrick)
Los Angeles Dodgers (Justin Wrobleski) @ St. Louis Cardinals (Sonny Gray)
New York Mets (Kodai Senga) @ Colorado Rockies (Ryan Rolison)
Seattle Mariners (Bryce Miller) @ Los Angeles Angels (Kyle Hendricks)
Baltimore Orioles (Dean Kremer) @ Athletics (JP Sears)
Atlanta Braves (Spencer Schwellenbach) @ San Francisco Giants (Hayden Bi

In [415]:
pitching_df, batting_df = get_rotowire_lineups()

In [416]:
import os
api_key = os.getenv("api_key")
print(api_key)

fa1d935c1d67073458a1f970bc434075


In [417]:
# Note that BBE = attempts = number of events
pitcher_barrels = statcast_pitcher_exitvelo_barrels(2025)

In [418]:
start_date = (date.today() + pd.DateOffset(days=-30)).strftime('%Y-%m-%d')
# start_date = '2025-05-01'
end_date = today
# end_date = '2025-06-03'
pitcher_ids = get_starting_ids(days_offset=days_offset)
pitcher_id_nos = [p[0] for p in pitcher_ids]
starters_data = {}
pitchers_arsenals = {}
pitcher_opponents = {}
pitcher_games = {}
for id, opposing_team, game_id in pitcher_ids:
    pitcher_data = statcast_pitcher(start_dt=start_date, end_dt=end_date,player_id=id)
    pitcher_opponents[id] = opposing_team
    starters_data[id] = pitcher_data.copy()
    pitcher_games[id] = game_id
    pitchers_arsenals[id] = {'R': get_pitch_arsenal(pitcher_data, 'R'), 'L': get_pitch_arsenal(pitcher_data, 'L')}

['Zack Littell', 'Edward Cabrera', 'Bailey Falter', 'Joe Ross', 'Michael Soroka', 'Patrick Corbin', 'Will Warren', 'Walker Buehler', 'Nick Lodolo', 'Eduardo Rodriguez', 'Tarik Skubal', 'Ben Brown', 'Logan Allen', 'Colton Gordon', 'Davis Martin', 'Seth Lugo', 'Bailey Ober', 'Paxton Schultz', 'Chad Patrick', 'Randy Vásquez', 'Sonny Gray', 'Justin Wrobleski', 'Ryan Rolison', 'Kodai Senga', 'Kyle Hendricks', 'Bryce Miller', 'JP Sears', 'Dean Kremer', 'Hayden Birdsong', 'Spencer Schwellenbach']
No identically matched names found! Returning the 5 most similar names.
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Playe

In [419]:
# pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['attempts'] > 50]
pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['player_id'].isin(pitcher_id_nos)].copy()
pitcher_barrels_copy['barrel_percentage'] = pitcher_barrels_copy['player_id'].apply(get_barrel_percentage, start_date=start_date, end_date=end_date)
pitcher_barrels_copy['left_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['left_pct'])
pitcher_barrels_copy['right_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['right_pct'])
pitcher_barrels_copy['opponent'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_opponents[x])
pitcher_barrels_copy['game_id'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_games.get(x))
for index, row in pitcher_barrels_copy.iterrows():
    if row['barrel_percentage']['total_pct'] == 0:
        pitcher_barrels_copy.at[index, 'left_barrel_pct'] = row['brl_percent'] / 100
        pitcher_barrels_copy.at[index, 'right_barrel_pct'] = row['brl_percent'] / 100
    
pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'opponent']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,opponent
1,"Littell, Zack",641793,777625,234,29,12.40,"{'left_barrels': 2, 'right_barrels': 9, 'total...",0.07,0.12,away
4,"Martin, Davis",663436,777618,223,23,10.30,"{'left_barrels': 5, 'right_barrels': 7, 'total...",0.10,0.14,away
6,"Kremer, Dean",665152,777613,221,20,9.00,"{'left_barrels': 4, 'right_barrels': 2, 'total...",0.10,0.04,home
10,"Schwellenbach, Spencer",680885,777619,215,16,7.40,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.05,home
18,"Lodolo, Nick",666157,777623,208,16,7.70,"{'left_barrels': 1, 'right_barrels': 5, 'total...",0.07,0.08,away
19,"Sears, JP",676664,777613,207,19,9.20,"{'left_barrels': 1, 'right_barrels': 10, 'tota...",0.08,0.14,away
26,"Ober, Bailey",641927,777615,203,16,7.90,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.05,0.14,away
42,"Falter, Bailey",663559,777621,194,14,7.20,"{'left_barrels': 0, 'right_barrels': 3, 'total...",0.00,0.05,away
48,"Hendricks, Kyle",543294,777614,190,18,9.50,"{'left_barrels': 6, 'right_barrels': 4, 'total...",0.12,0.09,away
52,"Vásquez, Randy",681190,777617,189,22,11.60,"{'left_barrels': 8, 'right_barrels': 5, 'total...",0.27,0.10,home


In [420]:
filtered_pitcher_barrels_copy = pitcher_barrels_copy[(pitcher_barrels_copy['left_barrel_pct'] > 0.07) | (pitcher_barrels_copy['right_barrel_pct'] > 0.07)].copy()
filtered_pitcher_barrels_copy['sides'] = filtered_pitcher_barrels_copy.apply(lambda row: get_sides(row['left_barrel_pct'], row['right_barrel_pct']),axis=1)
filtered_pitcher_barrels_copy['arsenal'] = filtered_pitcher_barrels_copy.apply(lambda row: pitchers_arsenals[row['player_id']], axis=1)
filtered_pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent']]

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent
1,"Littell, Zack",641793,777625,234,29,12.40,"{'left_barrels': 2, 'right_barrels': 9, 'total...",0.07,0.12,[R],"{'R': {'SL': 28.774928774928775, 'FF': 27.9202...",away
4,"Martin, Davis",663436,777618,223,23,10.30,"{'left_barrels': 5, 'right_barrels': 7, 'total...",0.10,0.14,"[L, R]","{'R': {'FC': 44.56928838951311, 'FF': 32.58426...",away
6,"Kremer, Dean",665152,777613,221,20,9.00,"{'left_barrels': 4, 'right_barrels': 2, 'total...",0.10,0.04,[L],"{'R': {'FC': 40.833333333333336, 'SI': 36.25},...",home
10,"Schwellenbach, Spencer",680885,777619,215,16,7.40,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.05,[L],"{'R': {'FF': 31.25, 'SL': 24.03846153846154}, ...",home
18,"Lodolo, Nick",666157,777623,208,16,7.70,"{'left_barrels': 1, 'right_barrels': 5, 'total...",0.07,0.08,"[L, R]","{'R': {'CH': 29.792746113989637, 'FF': 28.2383...",away
19,"Sears, JP",676664,777613,207,19,9.20,"{'left_barrels': 1, 'right_barrels': 10, 'tota...",0.08,0.14,"[L, R]","{'R': {'FF': 38.83248730964467, 'ST': 28.93401...",away
26,"Ober, Bailey",641927,777615,203,16,7.90,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.05,0.14,[R],"{'R': {'FF': 35.751295336787564, 'CH': 23.3160...",away
48,"Hendricks, Kyle",543294,777614,190,18,9.50,"{'left_barrels': 6, 'right_barrels': 4, 'total...",0.12,0.09,"[L, R]","{'R': {'SI': 40.0, 'CH': 30.952380952380953}, ...",away
52,"Vásquez, Randy",681190,777617,189,22,11.60,"{'left_barrels': 8, 'right_barrels': 5, 'total...",0.27,0.10,"[L, R]","{'R': {'SI': 38.839285714285715, 'FC': 27.6785...",home
55,"Patrick, Chad",694477,777617,186,12,6.50,"{'left_barrels': 5, 'right_barrels': 1, 'total...",0.11,0.03,[L],"{'R': {'FC': 50.0, 'SI': 27.536231884057973}, ...",away


Next step is
- For each batter in a lineup that a pitcher is facing, we want to get their barrel % against the pitcher's arsenal (the pitchers > 18%)
- Calculate the weighted pull air percentage (pull % * air %)
- Also eventually can add blast %, iso

In [421]:
pitcher_df = filtered_pitcher_barrels_copy.copy()
pitcher_df['hitter_team'] = pitcher_df.apply(lambda row: get_team(row['game_id'], row['opponent']), axis=1)
pitcher_df['opposing_lineup'] = pitcher_df.apply(lambda row: get_opposing_lineup(row['game_id'], row['opponent']), axis=1)
pitcher_df['opposing_lineup_names'] = pitcher_df.apply(lambda row: get_specific_lineup(row['hitter_team'], batting_df), axis=1)
pitcher_df['rotowire_ids'] = pitcher_df.apply(lambda row: get_player_ids(row['opposing_lineup_names'], player_id_mapping, row['hitter_team']), axis=1)
pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'opposing_lineup_names','hitter_team','rotowire_ids']]

Error fetching lineup for game 777619: list index out of range
Error fetching lineup for game 777613: list index out of range


ValueError: Player R. Laureano not found in player_id_mapping.csv

In [ ]:
hitter_barrels = statcast_batter_exitvelo_barrels(2025)

In [ ]:
def get_hitter_barrels(hitter_barrels, hitter_ids, start_date, end_date, pitch_mix, sides):
    if hitter_ids is None or len(hitter_ids) == 0:
        return {}
    hitter_barrels_copy = hitter_barrels[hitter_barrels['player_id'].isin(hitter_ids)].copy()
    hitter_barrels_copy['barrel_data'] = hitter_barrels_copy['player_id'].apply(hitter_barrel_percentage, start_date=start_date, end_date=end_date, pitch_mix=pitch_mix, sides=sides)
    hitter_barrels_copy['barrel_percentage'] = hitter_barrels_copy.apply(lambda row: get_barrel_pcts(row['barrel_data']), axis=1)
    hitter_barrels_copy['barrel_qual'] = hitter_barrels_copy['barrel_percentage'].apply(lambda x: check_barrel_pcts(x))
    # print(hitter_barrels_copy['barrel_qual'])
    filtered = hitter_barrels_copy[hitter_barrels_copy['barrel_qual'].apply(lambda x: x != [0,0,0,0])].copy()
    # print(filtered)
    if len(filtered) > 3:
        # Create a new column with just the first element
        filtered['barrel_qual_first'] = filtered['barrel_qual'].apply(lambda x: x[0])
        # Sort by that column and get top 3
        top_3 = filtered.nlargest(3, 'barrel_qual_first')
        # Drop the temporary column if you don't need it
        top_3 = top_3.drop('barrel_qual_first', axis=1)
        return dict(zip(zip(top_3['last_name, first_name'], top_3['player_id']), top_3['barrel_qual']))
    else:
        return dict(zip(zip(filtered['last_name, first_name'], filtered['player_id']), filtered['barrel_qual']))

In [ ]:
hitter_pitcher_df = pitcher_df.copy()
hitter_pitcher_df['hitter_barrels'] = hitter_pitcher_df.apply(lambda row: get_hitter_barrels(hitter_barrels, row['rotowire_ids'], start_date, end_date, row['arsenal'], row['sides']), axis=1)
hitter_pitcher_df['num_hitters'] = hitter_pitcher_df['hitter_barrels'].apply(lambda x: len(x))
hitter_pitcher_df['hitter_team'] = hitter_pitcher_df.apply(lambda row: get_team(row['game_id'], row['opponent']), axis=1)
hitter_pitcher_df['start_time'] = hitter_pitcher_df['game_id'].apply(lambda x: get_start_time(x))
hitter_pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'hitter_barrels', 'num_hitters', 'hitter_team', 'start_time']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup,hitter_barrels,num_hitters,hitter_team,start_time
1,"Littell, Zack",641793,777625,234,29,12.40,"{'left_barrels': 2, 'right_barrels': 9, 'total...",0.07,0.12,[R],"{'R': {'SL': 28.774928774928775, 'FF': 27.9202...",away,None,{},0,Marlins,
3,"Senzatela, Antonio",622608,777616,232,25,10.80,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.12,0.07,[L],"{'R': {'FF': 50.23474178403756, 'SL': 30.51643...",away,None,"{('Soto, Juan', 665742): [0.2112676056338028, ...",2,Mets,
4,"Martin, Davis",663436,777618,223,23,10.30,"{'left_barrels': 5, 'right_barrels': 7, 'total...",0.10,0.14,"[L, R]","{'R': {'FC': 44.56928838951311, 'FF': 32.58426...",away,None,"{('Witt Jr., Bobby', 677951): [0.175, 0.4, 0.0...",2,Royals,
6,"Kremer, Dean",665152,777613,221,20,9.00,"{'left_barrels': 4, 'right_barrels': 2, 'total...",0.10,0.04,[L],"{'R': {'FC': 40.833333333333336, 'SI': 36.25},...",home,None,"{('Soderstrom, Tyler', 691016): [0.171875, 0.3...",3,Athletics,
10,"Schwellenbach, Spencer",680885,777619,215,16,7.40,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.05,[L],"{'R': {'FF': 31.25, 'SL': 24.03846153846154}, ...",home,None,{},0,Giants,
18,"Lodolo, Nick",666157,777623,208,16,7.70,"{'left_barrels': 1, 'right_barrels': 5, 'total...",0.07,0.08,"[L, R]","{'R': {'CH': 29.792746113989637, 'FF': 28.2383...",away,None,{},0,D-backs,
26,"Ober, Bailey",641927,777615,203,16,7.90,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.05,0.14,[R],"{'R': {'FF': 35.751295336787564, 'CH': 23.3160...",away,None,"{('Springer, George', 543807): [0.234375, 0.34...",3,Blue Jays,
48,"Hendricks, Kyle",543294,777614,190,18,9.50,"{'left_barrels': 6, 'right_barrels': 4, 'total...",0.12,0.09,"[L, R]","{'R': {'SI': 40.0, 'CH': 30.952380952380953}, ...",away,None,"{('Raleigh, Cal', 663728): [0.3684210526315789...",3,Mariners,
52,"Vásquez, Randy",681190,777617,189,22,11.60,"{'left_barrels': 8, 'right_barrels': 5, 'total...",0.27,0.10,"[L, R]","{'R': {'SI': 38.839285714285715, 'FC': 27.6785...",home,None,{},0,Brewers,
55,"Patrick, Chad",694477,777617,186,12,6.50,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.04,[L],"{'R': {'FC': 52.036199095022624, 'SI': 26.6968...",away,None,"{('Sheets, Gavin', 657757): [0.141025641025641...",2,Padres,


In [ ]:
# Barrel percent, blast percent
all_keys = {}
teams = []
names = []
barrels = []
blasts = []
pull_air = []
iso = []
for index, row in hitter_pitcher_df.iterrows():
    for key in row['hitter_barrels']:
        all_keys[(row['hitter_team'], key)] = [f"{float(val):.3f}" for val in row['hitter_barrels'][key]]
        teams.append(row['hitter_team'])
        names.append(key)
        barrels.append(row['hitter_barrels'][key][0])
        blasts.append(row['hitter_barrels'][key][1])
        pull_air.append(row['hitter_barrels'][key][2])
        iso.append(row['hitter_barrels'][key][3])
sorted_batters = all_keys.items()
sorted_batters = sorted(sorted_batters, key=lambda x: x[1], reverse=True)
# for batter in sorted_batters:
#     print(batter)

chosen_hitters = pd.DataFrame({
    'team': teams,
    'name': names,
    'barrel_percent': barrels,
    'blast_percent': blasts,
    'pull_air_percent': pull_air,
    'iso': iso
})
chosen_hitters = chosen_hitters.sort_values(by='barrel_percent', ascending=False)
chosen_hitters

,team,name,barrel_percent,blast_percent,pull_air_percent,iso
10,Mariners,"(Raleigh, Cal, 663728)",0.37,0.53,0.53,0.57
26,Pirates,"(Reynolds, Bryan, 668804)",0.29,0.36,0.07,0.30
27,Pirates,"(Cruz, Oneil, 665833)",0.28,0.47,0.10,0.24
22,Yankees,"(Chisholm Jr., Jazz, 665862)",0.25,0.25,0.25,0.50
23,Yankees,"(Judge, Aaron, 592450)",0.25,0.48,0.20,0.39
16,White Sox,"(Benintendi, Andrew, 643217)",0.24,0.33,0.33,0.19
21,Nationals,"(Wood, James, 695578)",0.24,0.57,0.13,0.28
7,Blue Jays,"(Springer, George, 543807)",0.23,0.34,0.14,0.22
0,Mets,"(Soto, Juan, 665742)",0.21,0.44,0.10,0.24
12,Mariners,"(Tellez, Rowdy, 642133)",0.18,0.37,0.20,0.24


To do:
- Add ISO

In [ ]:
# hoskins = statcast_batter(start_dt='2025-05-01', end_dt='2025-06-03', player_id=663728)
# abs = hoskins[(hoskins['events'].notna()) & (~hoskins['events'].isin(['walk','hit_by_pitch','sac_fly','fielder_error']))]
# mapping = {'single': 1, 'double': 2, 'triple': 3, 'home_run': 4}
# abs['bases'] = abs['events'].map(lambda x: mapping.get(x, 0))
# abs_total = len(abs[abs['bases'] != 0])
# abs_avg = (abs['bases'].sum() - abs_total) / abs.shape[0]
# abs_avg

In [ ]:
chosen_hitters['player_id'] = chosen_hitters['name'].apply(lambda x: x[-1])
chosen_hitters['outcomes'] = chosen_hitters['player_id'].apply(lambda x: get_hitter_outcomes(x, yesterday, yesterday))
chosen_hitters


Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,team,name,barrel_percent,blast_percent,pull_air_percent,iso,player_id,outcomes
5,Pirates,"(Reynolds, Bryan, 668804)",0.29,0.36,0.07,0.32,668804,0
30,Mariners,"(Raleigh, Cal, 663728)",0.24,0.40,0.43,0.31,663728,0
18,Blue Jays,"(Springer, George, 543807)",0.23,0.34,0.15,0.23,543807,0
31,Mets,"(Soto, Juan, 665742)",0.22,0.42,0.11,0.25,665742,0
25,White Sox,"(Benintendi, Andrew, 643217)",0.21,0.28,0.31,0.18,643217,0
0,Cubs,"(Busch, Michael, 683737)",0.20,0.41,0.29,0.25,683737,0
6,Rangers,"(Langford, Wyatt, 694671)",0.18,0.46,0.14,0.17,694671,0
32,Mets,"(Alonso, Pete, 624413)",0.18,0.39,0.24,0.18,624413,0
19,Dodgers,"(Hernández, Teoscar, 606192)",0.18,0.32,0.15,0.15,606192,0
16,Blue Jays,"(Bichette, Bo, 666182)",0.17,0.29,0.14,0.22,666182,0


In [ ]:
len(chosen_hitters[chosen_hitters['outcomes'] > 1.5])

0

In [ ]:
len(chosen_hitters)

34

In [ ]:
len(chosen_hitters[chosen_hitters['outcomes'] > 0])

0